In [ ]:
import numpy as np
from scipy import sparse as spsp
import pandas as pd
import dgl
import torch

## Synthesize data

Please use the 'synthetic_data.ipynb' notebook to synthesize data in this tutorial.

## Construct a homogeneous graph

'edges.csv' stores a graph with multiple edge types. Although the graph has multiple edge types and node types, we can still construct it as a homogeneous graph. In this case, we create `DGLGraph` for this graph and stores node types and edge types as node data and edge data. In this graph, each node has two features.

In [ ]:
node_data = pd.read_csv('node_data.csv', header=None)
edges = pd.read_csv('edges.csv', header=None)

num_edges = edges.shape[0]
spm = spsp.coo_matrix((np.ones(num_edges), (edges[0], edges[1])))
g = dgl.DGLGraph(spm, readonly=True)
ndata1 = np.expand_dims(np.array(node_data[1]), 1)
ndata2 = np.expand_dims(np.array(node_data[2]), 1)
ndata = np.concatenate([ndata1, ndata2], 1)
g.ndata['feats'] = torch.tensor(ndata)
g.edata['rel'] = torch.tensor(np.array(edges[2]))

## Construct a heterogeneous graph with one node type

For the graph above, we can construct a heterogeneous graph with one node type. We first need to create an adjacency matrix for each edge type and call `heterograph` to create `DGLHeteroGraph`.

In [ ]:
node_data = pd.read_csv('node_data.csv', header=None)
edges = pd.read_csv('edges.csv', header=None)

rel = np.array(edges[2])
spm_dict = {}
src = edges[0][rel==0]
dst = edges[1][rel==0]
spm_dict['a', '0', 'a'] = spsp.coo_matrix((np.ones(len(src)), (src, dst)))

src = edges[0][rel==1]
dst = edges[1][rel==1]
spm_dict['a', '1', 'a'] = spsp.coo_matrix((np.ones(len(src)), (src, dst)))

src = edges[0][rel==2]
dst = edges[1][rel==2]
spm_dict['a', '2', 'a'] = spsp.coo_matrix((np.ones(len(src)), (src, dst)))

g = dgl.heterograph(spm_dict)

ndata1 = np.expand_dims(np.array(node_data[1]), 1)
ndata2 = np.expand_dims(np.array(node_data[2]), 1)
ndata = np.concatenate([ndata1, ndata2], 1)
g.ndata['feats'] = torch.tensor(ndata)

## Construct a heterogeneous graph with multiple node types

'edges1.csv' stores a graph with 3 edge types and 2 node types. We can construct a heterogeneous graph in a similar way as above. We first need to create an adjacency matrix for each edge type and call `heterograph` to create `DGLHeteroGraph`. Note: The third edge type connects nodes of type 'a' and 'b'.

After constructing the heterogeneous graph, we need to assign node data to each node type. We need to assign data to the nodes of type 'a' and 'b' separately.

In [ ]:
edges = pd.read_csv('edges1.csv', header=None)
node_data = pd.read_csv('node_data.csv', header=None)
node_data1 = pd.read_csv('node_data1.csv', header=None)

rel = np.array(edges[2])
spm_dict = {}
src = edges[0][rel==0]
dst = edges[1][rel==0]
spm_dict['a', '0', 'a'] = spsp.coo_matrix((np.ones(len(src)), (src, dst)))

src = edges[0][rel==1]
dst = edges[1][rel==1]
spm_dict['a', '1', 'a'] = spsp.coo_matrix((np.ones(len(src)), (src, dst)))

src = edges[0][rel==2]
dst = edges[1][rel==2]
spm_dict['a', '2', 'b'] = spsp.coo_matrix((np.ones(len(src)), (src, dst)))

g = dgl.heterograph(spm_dict)

ndata = np.concatenate([np.expand_dims(np.array(node_data[1]), 1),
                        np.expand_dims(np.array(node_data[2]), 1)], 1)
ndata1 = np.concatenate([np.expand_dims(np.array(node_data1[1]), 1),
                         np.expand_dims(np.array(node_data1[2]), 1),
                         np.expand_dims(np.array(node_data1[3]), 1)], 1)
g.nodes['a'].data['feats'] = torch.tensor(ndata)
g.nodes['b'].data['feats'] = torch.tensor(ndata1)

## Construct a heterogeneous graph with non-contiguous node Ids